Rzuty monetą: 

[https://docs.google.com/spreadsheets/d/1eI8uGEHNUZptv2wbNQdXLQTwQndGUstGFmtZ4VWNqWY/edit?usp=sharing]

In [1]:
import pandas as pd
URLCSV = "https://docs.google.com/spreadsheets/d/1eI8uGEHNUZptv2wbNQdXLQTwQndGUstGFmtZ4VWNqWY/export?format=csv"
dane = pd.read_csv(URLCSV)
print(dane.head())

   LP                                              rzuty  fake  score  \
0   1  OOOOROORRRRRRORORORROOOORORRRRORROOOROORRRROOO...     1    565   
1   2  OOROOORRORRROOROROORROROOOORORROOORRORRROROORO...     1    408   
2   3  ORORORORRORORORORRORORRROOOROOROOOORRRROOROORO...     1    513   
3   4  OORORROOORRROOORROOOORRROOORRORROORRORRORRORRR...     1    333   
4   5  OORORRROROROOORRORRORROOORORROOORORORROORROOOR...     1     88   

   pierwszy_O  liczba_serii  liczba_O  max_seria  runtest_z      sd10  \
0           1            49        49          6  -0.297627  0.128668   
1           1            57        53          5   1.145949  0.082327   
2           1            56        52          9   0.922152  0.139841   
3           1            53        49          4   0.305671  0.087560   
4           1            64        48          3   2.532898  0.078881   

   sum_2max  sum_3max  sum_4max  sum_5max  sum_6max  sum_7max  count_1  
0        11        15        19        23        

In [5]:
import numpy as np
mask = dane['fake']==1
print("Średnia maksymalna długość serii dla udawanych rzutów monetą. ")
np.mean(dane['max_seria'][mask])

Średnia maksymalna długość serii dla udawanych rzutów monetą. 


np.float64(4.866666666666666)

In [6]:
print("Średnia maksymalna długość serii dla generowanych rzutów monetą. ")
np.mean(dane['max_seria'][~mask])

Średnia maksymalna długość serii dla generowanych rzutów monetą. 


np.float64(6.907)

Przedział ufności dla średniej maksymalnej serii:

- dla rzutów wymyślonych przez studentów

- dla rzutów generowanych z generatora liczb losowych

In [8]:
import numpy as np
from scipy import stats

# Przedział ufności (dane wymyślone przez Studentów)
data = dane['max_seria'][mask]
test_result = stats.ttest_1samp(data, popmean=np.mean(data))
conf_int = test_result.confidence_interval(0.95)
print(conf_int)

ConfidenceInterval(low=np.float64(4.26430124154635), high=np.float64(5.469032091786983))


In [9]:
# Przedział ufności (dane wygenerowane)
data = dane['max_seria'][~mask]
test_result = stats.ttest_1samp(data, popmean=np.mean(data))
conf_int = test_result.confidence_interval(0.95)
print(conf_int)

ConfidenceInterval(low=np.float64(6.795377667597976), high=np.float64(7.018622332402024))


In [12]:
x = dane['max_seria']
y = dane['sd10']
correlation_test = stats.pearsonr(x, y)
confidence_interval = correlation_test.confidence_interval(0.95)
print('Test:\n\n', correlation_test, '\n\n Confidence interval: \n\n', confidence_interval)

Test:

 PearsonRResult(statistic=np.float64(0.4834710849886912), pvalue=np.float64(1.897535886194581e-61)) 

 Confidence interval: 

 ConfidenceInterval(low=np.float64(0.43524140922139465), high=np.float64(0.528933839108973))
